In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import json

In [18]:
# Configuration de l'affichage pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [19]:
# Liste des villes
cities = ["Paris", "Lyon", "Marseille", "Toulouse", "Bordeaux", "Nantes", "Nice"]

In [20]:
class Numbeo:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

    def extract_cost_of_living(self, cities):
        """Extraction des données brutes du coût de la vie"""
        raw_data = []
        for city in cities:
            try:
                print(f"Extraction des données de coût de la vie pour {city}...")
                url = f"https://www.numbeo.com/cost-of-living/in/{city}?displayCurrency=EUR"
                response = requests.get(url, headers=self.headers)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.text, 'html.parser')
                tables = soup.find_all('table', class_='data_wide_table')
                
                data = {'City': city}
                for table in tables:
                    for row in table.find_all('tr'):
                        cells = row.find_all('td')
                        if len(cells) >= 2:
                            item_name = cells[0].text.strip()
                            price_text = cells[1].text.strip()
                            data[item_name] = price_text
                
                raw_data.append(data)
                time.sleep(2)
            except Exception as e:
                print(f"Erreur extraction coût de la vie {city}: {str(e)}")
        
        # Sauvegarde des données brutes
        with open('data/extract/cost_of_living_raw.json', 'w', encoding='utf-8') as f:
            json.dump(raw_data, f, ensure_ascii=False, indent=4)
        
        df = pd.DataFrame(raw_data)
        df.to_csv('data/extract/cost_of_living_raw.csv', index=False)
        
        return raw_data

    def extract_health(self, cities):
        """Extraction des données brutes de santé"""
        raw_data = []
        for city in cities:
            try:
                print(f"Extraction des données de santé pour {city}...")
                url = f"https://www.numbeo.com/health-care/in/{city}"
                response = requests.get(url, headers=self.headers)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.text, 'html.parser')
                
                data = {'City': city}
                
                # Extraction de l'index de santé général
                health_index = soup.find('td', string=re.compile('Health Care Index:'))
                if health_index and health_index.find_next_sibling():
                    data['Health Care Index'] = health_index.find_next_sibling().text.strip()
                
                # Extraction des composants de santé depuis la table
                table = soup.find('table', {'class': 'table_builder_with_value_explanation data_wide_table'})
                if table:
                    for row in table.find_all('tr')[1:]:  # Skip header row
                        cols = row.find_all('td')
                        if len(cols) >= 3:
                            component = cols[0].text.strip()
                            value = cols[2].text.strip()
                            data[component] = value
                
                raw_data.append(data)
                time.sleep(2)
                
            except Exception as e:
                print(f"Erreur extraction santé {city}: {str(e)}")
        
        # Sauvegarde des données brutes
        with open('data/extract/health_raw.json', 'w', encoding='utf-8') as f:
            json.dump(raw_data, f, ensure_ascii=False, indent=4)
        
        df = pd.DataFrame(raw_data)
        df.to_csv('data/extract/health_raw.csv', index=False)
        
        return raw_data

In [21]:
class DataCleaner:
    def clean_cost_of_living(self, raw_data):
        """Nettoyage des données du coût de la vie et conversion des types"""
        cleaned_data = []
        for city_data in raw_data:
            cleaned_city_data = {'City': city_data['City']}
            for key, value in city_data.items():
                if key != 'City':
                    # Nettoyer et convertir en type approprié
                    cleaned_city_data[key] = self.convert_to_numeric(self.clean_price(value))
            cleaned_data.append(cleaned_city_data)

        # Sauvegarde des données nettoyées
        with open('data/transform/cost_of_living_cleaned.json', 'w', encoding='utf-8') as f:
            json.dump(cleaned_data, f, ensure_ascii=False, indent=4)
        
        df = pd.DataFrame(cleaned_data)
        df.to_csv('data/transform/cost_of_living_cleaned.csv', index=False)
        
        return cleaned_data

    def clean_health(self, raw_data):
        """Nettoyage des données de santé et séparation des valeurs numériques"""
        cleaned_data = []
        for city_data in raw_data:
            cleaned_city_data = {'City': city_data['City']}
            for key, value in city_data.items():
                if key != 'City':
                    try:
                        # Remplacer \n par un espace et nettoyer
                        value = value.replace('\n', ' ').strip()
                        
                        # Séparer la partie numérique et textuelle
                        if value.split(' ')[0].replace('.', '').isdigit():
                            number = float(value.split(' ')[0])
                            rating = ' '.join(value.split(' ')[1:])  # Partie textuelle (High, Very High, etc.)
                            
                            # Créer deux colonnes distinctes pour la valeur numérique et le rating
                            cleaned_city_data[f"{key}_value"] = number
                            if rating:  # Si le rating existe
                                cleaned_city_data[f"{key}_rating"] = rating
                        else:
                            cleaned_city_data[key] = value
                    except (ValueError, AttributeError):
                        cleaned_city_data[key] = value

            cleaned_data.append(cleaned_city_data)

        # Sauvegarde des données nettoyées
        with open('data/transform/health_cleaned.json', 'w', encoding='utf-8') as f:
            json.dump(cleaned_data, f, ensure_ascii=False, indent=4)
        
        df = pd.DataFrame(cleaned_data)
        df.to_csv('data/transform/health_cleaned.csv', index=False)
        
        return cleaned_data

    @staticmethod
    def clean_price(price_text):
        """Nettoyage des prix - suppression des symboles d'euro"""
        try:
            price = price_text.replace(' ', '').strip()
            if '€' in price:
                price = price.replace('€', '').strip()
            price = price.replace(',', '.')
            
            if price and price.replace('.', '').isdigit():
                return price  # Retourne juste la chaîne nettoyée sans formater
            return price_text.strip().replace(' €', '')
        except:
            return price_text.strip().replace(' €', '')

    @staticmethod
    def convert_to_numeric(value_str):
        """Convertit une chaîne en un type numérique approprié (float ou int)"""
        try:
            # Tenter de convertir en float
            value_float = float(value_str)
            
            # Si c'est un entier (pas de partie décimale), convertir en int
            if value_float.is_integer():
                return int(value_float)
            else:
                return value_float
        except (ValueError, TypeError):
            # Si la conversion échoue, retourner la valeur originale
            return value_str

In [22]:
class DataStorage:
    def save_final_data(self, cost_data, health_data):
        """Sauvegarde des données finales"""
        # Préparation des DataFrames
        cost_df = pd.DataFrame(cost_data).set_index('City')
        health_df = pd.DataFrame(health_data).set_index('City')
        
        # Sauvegarde des données finales
        cost_df.to_csv('data/load/cost_of_living_final.csv')
        health_df.to_csv('data/load/health_final.csv')
        
        # Sauvegarde en JSON
        with open('data/load/cost_of_living_final.json', 'w', encoding='utf-8') as f:
            json.dump(cost_data, f, ensure_ascii=False, indent=4)
        
        with open('data/load/health_final.json', 'w', encoding='utf-8') as f:
            json.dump(health_data, f, ensure_ascii=False, indent=4)
        
        return cost_df, health_df


In [23]:
def create_directories():
    """Création des répertoires nécessaires"""
    import os
    directories = ['data/extract', 'data/transform', 'data/load']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

In [24]:
def main():
    # Liste des villes
    cities = ["Paris", "Lyon", "Marseille", "Toulouse", "Bordeaux", "Nantes", "Nice"]
    
    # Création des répertoires
    create_directories()
    
    # Initialisation des classes
    numbeo = Numbeo()
    cleaner = DataCleaner()
    storage = DataStorage()
    
    # 1. Extraction
    print("1. Extraction des données...")
    cost_raw_data = numbeo.extract_cost_of_living(cities)
    health_raw_data = numbeo.extract_health(cities)
    
    # 2. Transformation
    print("\n2. Nettoyage des données...")
    cost_cleaned_data = cleaner.clean_cost_of_living(cost_raw_data)
    health_cleaned_data = cleaner.clean_health(health_raw_data)
    
    # 3. Chargement
    print("\n3. Sauvegarde des données finales...")
    cost_df, health_df = storage.save_final_data(cost_cleaned_data, health_cleaned_data)
    
    print("\nAperçu des données de coût de la vie :")
    display(cost_df)
    
    print("\nAperçu des données de santé :")
    display(health_df)

if __name__ == "__main__":
    main()

1. Extraction des données...
Extraction des données de coût de la vie pour Paris...
Extraction des données de coût de la vie pour Lyon...
Extraction des données de coût de la vie pour Marseille...
Extraction des données de coût de la vie pour Toulouse...
Extraction des données de coût de la vie pour Bordeaux...
Extraction des données de coût de la vie pour Nantes...
Extraction des données de coût de la vie pour Nice...
Extraction des données de santé pour Paris...
Extraction des données de santé pour Lyon...
Extraction des données de santé pour Marseille...
Extraction des données de santé pour Toulouse...
Extraction des données de santé pour Bordeaux...
Extraction des données de santé pour Nantes...
Extraction des données de santé pour Nice...

2. Nettoyage des données...

3. Sauvegarde des données finales...

Aperçu des données de coût de la vie :


,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),Water (0.33 liter bottle),"Milk (regular), (1 liter)",Loaf of Fresh White Bread (500g),"Rice (white), (1kg)",Eggs (regular) (12),Local Cheese (1kg),Chicken Fillets (1kg),Beef Round (1kg) (or Equivalent Back Leg Red Meat),Apples (1kg),Banana (1kg),Oranges (1kg),Tomato (1kg),Potato (1kg),Onion (1kg),Lettuce (1 head),Water (1.5 liter bottle),Bottle of Wine (Mid-Range),Domestic Beer (0.5 liter bottle),Cigarettes 20 Pack (Marlboro),One-way Ticket (Local Transport),Monthly Pass (Regular Price),Taxi Start (Normal Tariff),Taxi 1km (Normal Tariff),Taxi 1hour Waiting (Normal Tariff),Gasoline (1 liter),Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car),Toyota Corolla Sedan 1.6l 97kW Comfort (Or Equivalent New Car),"Basic (Electricity, Heating, Cooling, Water, Garbage) for 85m2 Apartment",Mobile Phone Monthly Plan with Calls and 10GB+ Data,"Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)","Fitness Club, Monthly Fee for 1 Adult",Tennis Court Rent (1 Hour on Weekend),"Cinema, International Release, 1 Seat","Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child","International Primary School, Yearly for 1 Child",1 Pair of Jeans (Levis 501 Or Similar),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Paris,15,65.0,12.0,7.0,3.42,3.87,3.72,2.94,1.31,1.90,2.69,4.36,22.06,12.25,19.49,3.12,2.06,3.11,3.58,2.34,2.59,1.53,0.75,8.0,2.27,12.50,2.5,87.0,8,1.96,48.0,1.88,30.870.00,31.906.25,223.05,15.02,32.41,34.14,17.68,13,711.11,15.500.00,101.59,42.33,108.00,140.37,1.365.64,1.018.90,3.082.76,2.080.00,12.260.87,8.603.45,3.003.00,3.66
Lyon,15,65.0,11.0,7.0,2.45,3.45,2.62,1.62,1.31,2.41,2.38,3.64,21.20,12.94,23.74,2.49,2.14,3.28,3.46,2.44,2.35,1.36,0.75,8.0,2.34,12.00,2.0,72.0,6.5,1.8,40.0,1.78,32.395.00,32.012.50,246.03,21.12,32.36,39.36,20.00,12,560,10.333.33,100.43,37.83,101.71,133.00,928.69,643.77,1.742.50,1.104.44,6.126.86,3.899.67,2.601.42,3.76
Marseille,15,60.0,10.0,6.0,3.50,3.01,2.60,1.68,1.24,1.92,2.90,3.67,22.00,11.50,23.67,2.92,2.05,3.40,3.28,2.52,2.25,2.10,1.23,7.5,3.62,12.00,1.8,40.0,5,3,25.0,1.83,32.395.00,30.723.33,161.71,25.60,31.11,40.00,18.60,13,705,8.625.00,91.67,39.83,99.17,124.00,732,600,1.287.50,1.216.67,4.999.88,3.128.50,2.019.61,3.83
Toulouse,14,60.0,11.0,6.5,2.67,2.83,2.37,1.61,1.25,2.00,2.64,3.61,22.00,12.56,15.76,2.83,2.68,2.80,3.06,2.37,2.75,1.56,0.81,7.0,3.42,12.75,1.8,56.0,2.6,1.3,35.2,1.77,32.395.00,34.275.00,164.76,19.34,31.75,35.88,23.00,12,741.94,16.000.00,99.17,35.66,104.14,131.67,748.33,578.57,1.393.75,1.071.43,4.514.67,3.118.14,2.587.30,3.71
Bordeaux,18,60.0,10.5,7.0,3.75,3.50,2.41,1.34,1.07,1.85,2.75,4.32,13.25,13.42,19.60,2.88,2.00,3.36,3.95,2.08,2.40,1.60,0.60,9.0,3.58,11.50,1.8,42.2,?,?,42.0,1.81,32.395.00,30.625.00,219.71,21.23,28.23,32.41,25.00,12,729,8.132.00,88.75,39.14,85.62,124.83,771.67,613.33,1.387.50,1.043.75,5.452.88,4.383.33,2.327.67,3.79
Nantes,15,57.5,10.5,7.0,2.83,3.30,2.41,1.25,1.04,1.85,2.71,3.00,13.25,13.42,18.33,2.50,1.70,2.25,2.67,1.50,1.93,1.38,0.50,6.0,3.02,11.00,1.8,57.0,5,2.48,32.1,1.77,32.395.00,30.625.00,180.32,20.92,23.60,27.15,10.00,12,729,8.132.00,88.75,27.50,82.50,124.83,730.83,630.83,1.760.00,1.266.67,4.624.24,3.382.29,2.113.14,3.62
Nice,20,75.0,10.5,7.0,2.50,3.10,3.06,2.31,1.37,1.58,2.03,3.93


Aperçu des données de santé :


,Skill and competency of medical staff_value,Skill and competency of medical staff_rating,Speed in completing examinations and reports_value,Speed in completing examinations and reports_rating,Equipment for modern diagnosis and treatment_value,Equipment for modern diagnosis and treatment_rating,Accuracy and completeness in filling out reports_value,Accuracy and completeness in filling out reports_rating,Friendliness and courtesy of the staff_value,Friendliness and courtesy of the staff_rating,Satisfaction with responsiveness (waitings) in medical institutions_value,Satisfaction with responsiveness (waitings) in medical institutions_rating,Satisfaction with cost to you_value,Satisfaction with cost to you_rating,Convenience of location for you_value,Convenience of location for you_rating
City,,,,,,,,,,,,,,,,
Paris,81.75,Very High,74.74,High,86.24,Very High,77.67,High,67.29,High,58.20,Moderate,81.58,Very High,83.24,Very High
Lyon,81.06,Very High,75.00,High,86.72,Very High,78.33,High,67.19,High,54.55,Moderate,80.30,Very High,81.25,Very High
Marseille,86.76,Very High,65.62,High,88.24,Very High,83.82,Very High,75.00,High,72.06,High,91.18,Very High,87.50,Very High
Toulouse,85.34,Very High,81.48,Very High,90.74,Very High,82.41,Very High,82.76,Very High,72.32,High,89.66,Very High,81.25,Very High
Bordeaux,90.38,Very High,82.69,Very High,84.62,Very High,84.62,Very High,80.77,Very High,72.92,High,86.54,Very High,88.46,Very High
Nantes,91.67,Very High,78.33,High,86.67,Very High,83.93,Very High,76.67,High,58.33,Moderate,82.14,Very High,75.00,High
Nice,89.00,Very High,82.00,Very High,91.30,Very High,85.23,Very High,79.00,High,68.75,High,91.67,Very High,90.91,Very High
